# Cotiss Takehome Project

## Overview

In Australia and New Zealand alone there are over 40,000 government contracts posted each year. Most contracts have common meta data such as the date they are published, their associated category and the region in which the good/service is being procured. In addition, they contain free text that describes the details of the tender.

At Cotiss, part of our mission is to help businesses find those opportunities that are relevant for them. In order to make these contracts accessible to SMEs, it should take minimal resources to find relevant opportunities. One of the key components in this is making them searchable. This means that the content of the tender can be processed, so relevant businesses can be notified.

A key issue in searching for listings is that, often, those searching don't know what phrases to search for. An improvement for plaintext search is an autocomplete search feature that prompts the user on what they should search. You have probably experienced this when doing a Google search.

## The task

For this task, we have collated a file called _listings.json_ containing the content of thousands of public tenders posted through Government portals. Your job is to transform the data into a structure that is easier to search.

We need to improve our capabilities for our users to quickly locate tenders that matter to them. Currently our users are having a hard time to filter out irrelevant tenders and the locating tenders for their business.

User story: As a user I want to be able to effectively search for tenders that relate to my business
Acceptance Criteria
- Can efficiently search thousands of tenders
- Only shows tenders relating to the search phrase
- Recommends what phrases to search for
- Filters results by meta data

We are not asking you to implement the search component itself. The task is to create a model that will make it simpler for a search API to perform the above tasks. This project is open ended and designed to provide a platform to show your creativity, coding and problem solving skills. There is also no strict requirement on what the final solution should look like. There are no right or wrong answers so don't overthink it - just state your assumptions and justification for the model you think works best. If you have any questions, please reach out to matthew.oh@cotiss.com and ask away - we're here to help!

Your approach may also include external services in addition to python. However, if you do so, please state what service was used and how you used it. An example might be using AWS Comprehend to extract key phrases. There are also many examples for this type of application online. If you use example code, please include the required attribution.

## Output

- Once the model has been produced, it should show the output when run on the file provided. 
- Graphical analysis in order to interprets the results.
- Suggestions on how this data could be combined with the search input to produce results. For example: "We recommend using n-gram to match a searched phrase, the data returned will be a list of matched tenders and relevant meta-data".


## Submission

Before you start this project, please fork the repository and add it to your GitHub. Everything must be documented below this description, including the code. From the time you have received the project, you will have 7 days to complete it. Once you have completed it, reach out to matthew.oh@cotiss.com. We will aim to get back to you within a couple of days. Last of all, please timebox this excerice to 2-3 hours. We value your time, so don't expect a near perfect solution.